In [17]:
import ipyleaflet as L
import numpy as np
from dataloader import TripsLoader
from sklearn.cluster import KMeans
from tqdm import tqdm
import pandas as pd

In [18]:
loader = TripsLoader('04-26-2022')

In [19]:
#Dumper statistics
nb_of_dumpers = 0
km_total = 0
quantity_total = 0
duration_total = 0
dumper_dict = {}

for machine_key in tqdm(loader._machines.keys()):
    machine = loader._machines[machine_key]
    if machine.machine_type=='Dumper':
        nb_of_dumpers += 1
        km_total += machine.total_length
        duration_total += machine.total_duration
        dumper_dict[machine.machine_id] = {
            'nb_of_trips': len(machine.trips),
            'total_length': machine.total_length,
            'total_quantity': machine.total_quantity,
            'total_duration': machine.total_duration,
            'nb_stone_trips': len(machine.stone_trips),
            'nb_soil_trips': len(machine.soil_trips),
            'nb_equip_trips': len(machine.equipment_trips),
            'nb_four_trips': len(machine.four_trips)
        }


100%|██████████| 17/17 [00:00<00:00, 20.97it/s]


In [20]:
nb_of_dumpers

2

In [21]:
dumper_pd = pd.DataFrame.from_dict(dumper_dict, orient='index')

In [22]:
dumper_pd

,nb_of_trips,total_length,total_quantity,total_duration,nb_stone_trips,nb_soil_trips,nb_equip_trips,nb_four_trips
37,15,51.917962,600.0,1431.484050,0,0,15,0
19,49,33.686969,1078.0,704.056683,43,6,0,0


In [23]:
all_positions_of_dumpers = []

In [24]:
for machine_key in loader._machines.keys():
    machine = loader._machines[machine_key]
    if machine.machine_type=='Dumper':
        for trip in machine.trips:
            for pos in trip.positions:
                all_positions_of_dumpers.append((pos.lat, pos.lon))

coordinates_array = np.array(all_positions_of_dumpers)

In [25]:
# Create a map centered at the mean of all coordinates, with heatmap
map_center = np.mean(coordinates_array, axis=0)
m = L.Map(center=(map_center[0], map_center[1]), zoom=10)

# Add markers for each cluster center to the map
#for center in cluster_centers:
#    marker = L.Marker(location=(center[0], center[1]))
#    m.add_layer(marker)
heatmap = L.Heatmap(locations=all_positions_of_dumpers,radius=20)
m.add_layer(heatmap)

# Display the map
m

#Can see two clear clusters in the map, maybe three. Can use K-means to find center of them.

Map(center=[59.9453250848679, 10.394715934433638], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [46]:
coordinates_array

array([[59.93960932, 10.42080402],
       [59.93960264, 10.42082036],
       [59.93960267, 10.42086756],
       ...,
       [59.95218607, 10.37476961],
       [59.9521941 , 10.3748373 ],
       [59.95224598, 10.3747533 ]])

In [47]:
# Fit the KMeans model with the optimal K value
kmeans = KMeans(n_clusters=2, random_state=42, n_init='auto')
kmeans.fit(coordinates_array)

# Get the coordinates of the cluster centers for the optimal K value
cluster_centers = kmeans.cluster_centers_

In [48]:
# Create a map centered at the mean of all coordinates, with heatmap
map_center = np.mean(coordinates_array, axis=0)
m = L.Map(center=(map_center[0], map_center[1]), zoom=10)

# Add markers for each cluster center to the map
for center in cluster_centers:
    marker = L.Marker(location=(center[0], center[1]))
    m.add_layer(marker)
heatmap = L.Heatmap(locations=all_positions_of_dumpers,radius=20)
m.add_layer(heatmap)

# Display the map
m

Map(center=[59.94312237689612, 10.405949415768172], controls=(ZoomControl(options=['position', 'zoom_in_text',…